<a href="https://colab.research.google.com/github/EdnilsonRosa/Data-Science/blob/master/Avalia%C3%A7%C3%A3o%20Big%20Data%202019%2005%2024%2016%3A00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avaliação Big Data - Phyton 2

## Inicializando o ambiente e carregando as bibliotecas

In [1]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.2/spark-2.4.2-bin-hadoop2.7.tgz
!tar xf spark-2.4.2-bin-hadoop2.7.tgz
!pip install -q findspark

!java -version

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-2.4.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Chamada da biblioteca

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

### Inicializaçaõ da Sessão do Spark - 

spark = SparkSession.builder \
                    .appName("Tutorial PySpark SQL") \
                    .config("spark.some.config.option", "some-value") \
                    .getOrCreate()

openjdk version "11.0.3" 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [2]:
!pip install jellyfish
import jellyfish

     |████████████████████████████████| 133kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/9c/6f/33/92bb9a4b4562a60ba6a80cedbab8907e48bc7a8b1f369ea0ae
Successfully built jellyfish


In [0]:
import pandas as pd

## Montando o Drive e carregando o arquivo

In [4]:
### Montagem de drive do Google
### Será solicitada autorização
### O Colab tem como raiz uma pasta 'content'

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dfa = spark.read.csv('gdrive/My Drive/Colab Notebooks/base_sintetica_ascii_a.csv', header=True, sep=',')

In [0]:
dfb = spark.read.csv('gdrive/My Drive/Colab Notebooks/base_sintetica_ascii_b.csv', header=True, sep=',')

In [7]:
dfa.count()

540

In [8]:
dfb.count()

143

In [9]:
dfa.show(5)

+---+--------------+---------+---------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+---------+--------------+----+
| id|        bairro|      cep|   cidade|           cpf| data_nasc|               email|            endereco|estado|idade|                 mae|                nome|numero|    signo|tipo_sanguineo|_c15|
+---+--------------+---------+---------+--------------+----------+--------------------+--------------------+------+-----+--------------------+--------------------+------+---------+--------------+----+
|  0|   Vila Sarney|65090-677| Sao Luis|097.627.958-49|10/09/1948|vicenteolivermont...|2a Travessa do Ri...|    MA|   71|     Louise Beatriz |Vicente Oliver Mo...|   738|   Virgem|           AB-|null|
|  1|Vila Sao Jorge|75044-220| Anapolis|685.096.335-09|24/04/1995|miguelbernardogon...|  Praca Joao Salomao|    GO|   24|     Raquel Barbara |Miguel Bernardo G...|   805|    Touro|            A-|n

In [10]:
dfb.limit(5).toPandas()

,id,cep,cidade,cpf,data_nasc,endereco,estado,mae,nome,tipo_sanguineo
0,01,69304-350,Boa Vista,506.865.202-46,22/11/1965,Avenida Mario Homem de Melo,RR,Isabelle Marcia Antonella,Felipe Bruno dos Santos,AB-
1,02,78132-360,Varzea Grande,325.828.867-42,None,Rua Rio Cuiaba,MT,Alana Isadora,Marcio Heitor Silveira,AB-
2,03,88810-336,Criciuma,284.536.218-88,23/09/1961,Rua Romeu Lopes de Carvalho,SC,Caroline Gabriela,Theo Mario Victor Barbosa,A-
3,04,60874-405,Fortaleza,390.375.925-20,11/05/1981,None,CE,Tereza Bruna,Pietro Henrique de Paula,AB-
4,05,79106-140,Campo Grande,756.005.387-40,14/12/1941,Rua Pato Branco,MS,Alice Leticia,Kaique Thiago de Paula,A-


In [11]:
dfa.printSchema()
dfb.printSchema()

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- _c15: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)



## Tratamento de Atributo Missing


In [12]:
dfa.filter(F.col('nome').isNull()).count()

0

In [13]:
dfb.filter(F.col('nome').isNull()).count()

2

In [0]:
# Trocando nomes nulos por espaço em branco (os valores anteriores são sobrescritos ao utilizar o mesmo nome de coluna)
dfb = dfb.withColumn('nome', F.when(F.col('nome').isNull()," ").otherwise(F.col('nome')))

In [15]:
dfb.filter(F.col('nome').isNull()).count()

0

## **Outros Tratamentos**

In [0]:
# Guardando data frames originais sem pré=processamento
dfa_o = dfa
dfb_o = dfb

In [0]:
# Eliminação de possíveis espaços duplos nos nomes
dfa = dfa.withColumn("nome", F.regexp_replace(F.col("nome"), '\s+', ' '))
dfb = dfb.withColumn("nome", F.regexp_replace(F.col("nome"), '\s+', ' '))
dfa = dfa.withColumn("mae", F.regexp_replace(F.col("mae"), '\s+', ' '))
dfb = dfb.withColumn("mae", F.regexp_replace(F.col("mae"), '\s+', ' '))
dfa = dfa.withColumn("cidade", F.regexp_replace(F.col("cidade"), '\s+', ' '))
dfb = dfb.withColumn("cidade", F.regexp_replace(F.col("cidade"), '\s+', ' '))
dfa = dfa.withColumn("endereco", F.regexp_replace(F.col("endereco"), '\s+', ' '))
dfb = dfb.withColumn("endereco", F.regexp_replace(F.col("endereco"), '\s+', ' '))

# Substituição de possiveis caracteres especiais por espaço em branco
dfa = dfa.withColumn("nome", F.regexp_replace(F.col("nome"), r'[^a-zA-Z ]+', ' '))
dfb = dfb.withColumn("nome", F.regexp_replace(F.col("nome"), r'[^a-zA-Z ]+', ' '))
dfa = dfa.withColumn("mae", F.regexp_replace(F.col("mae"), r'[^a-zA-Z ]+', ' '))
dfb = dfb.withColumn("mae", F.regexp_replace(F.col("mae"), r'[^a-zA-Z ]+', ' '))
dfa = dfa.withColumn("cidade", F.regexp_replace(F.col("cidade"), r'[^a-zA-Z ]+', ' '))
dfb = dfb.withColumn("cidade", F.regexp_replace(F.col("cidade"), r'[^a-zA-Z ]+', ' '))
dfa = dfa.withColumn("endereco", F.regexp_replace(F.col("endereco"), r'[^a-zA-Z ]+', ' '))


## Renomeando colunas para o linkage (necessita nomes diferentes)

In [0]:
dfb = dfb.withColumnRenamed('id','id_b')
dfb = dfb.withColumnRenamed('cep','cep_b')
dfb = dfb.withColumnRenamed('cidade','cidade_b')
dfb = dfb.withColumnRenamed('cpf','cpf_b')
dfb = dfb.withColumnRenamed('data_nasc','data_nasc_b')
dfb = dfb.withColumnRenamed('endereco','endereco_b')
dfb = dfb.withColumnRenamed('estado','estado_b')
dfb = dfb.withColumnRenamed('mae','mae_b')
dfb = dfb.withColumnRenamed('nome','nome_b')
dfb = dfb.withColumnRenamed('tipo_sanguineo','tipo_sanguineo_b')

In [19]:
dfa.printSchema()
dfb.printSchema()

root
 |-- id: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- email: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- signo: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- _c15: string (nullable = true)

root
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullabl

## Fazendo Linkage

In [0]:
# CrossJoin somente com colunas que serão utilizadas na análise

dfl = dfa.select('id','cep','cidade','data_nasc','endereco','estado','idade','mae','nome','tipo_sanguineo','cpf').crossJoin( \
      dfb.select('id_b','cep_b','cidade_b','data_nasc_b','endereco_b','estado_b','mae_b','nome_b','tipo_sanguineo_b','cpf_b'))

In [21]:
dfl.limit(5).show()

+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+
| id|      cep|  cidade| data_nasc|            endereco|estado|idade|            mae|                nome|tipo_sanguineo|           cpf|id_b|    cep_b|     cidade_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|         cpf_b|
+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+
|  0|65090-677|Sao Luis|10/09/1948| a Travessa do Ri...|    MA|   71|Louise Beatriz |Vicente Oliver Mo...|           AB-|097.627.958-49|  01|69304-350|   

## Criação de Funções para extração do primeiro e último nome

In [0]:
def criaPrimeiroNome(col):
  return col.split(' ') [0]
udf_criaPrimeiroNome = F.udf(criaPrimeiroNome, StringType())

In [0]:
def criaUltimoNome(col):
  return col.split(' ') [-1]
udf_criaUltimoNome = F.udf(criaUltimoNome, StringType())

## Criação das Colunas dos Primeiros e Ultimos Nomes

In [0]:
dfl = dfl.withColumn('primeiro_nome_a', udf_criaPrimeiroNome(F.col('nome')))
dfl = dfl.withColumn('primeiro_nome_b', udf_criaPrimeiroNome(F.col('nome_b')))
dfl = dfl.withColumn('ultimo_nome_a', udf_criaUltimoNome(F.col('nome')))
dfl = dfl.withColumn('ultimo_nome_b', udf_criaUltimoNome(F.col('nome_b')))

In [25]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

In [28]:
dfl.show(5)

+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+---------------+---------------+-------------+-------------+
| id|      cep|  cidade| data_nasc|            endereco|estado|idade|            mae|                nome|tipo_sanguineo|           cpf|id_b|    cep_b|     cidade_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|         cpf_b|primeiro_nome_a|primeiro_nome_b|ultimo_nome_a|ultimo_nome_b|
+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+---------------+---------------+---

## Criação das colunas com nomes fonéticos

In [0]:
def criaMetaphone(col):
  return jellyfish.metaphone(col)
udf_criaMetaphone = F.udf(criaMetaphone, StringType())

In [0]:
dfl = dfl.withColumn('primeiro_nome_fonetico_a', udf_criaMetaphone(F.col('primeiro_nome_a')))
dfl = dfl.withColumn('primeiro_nome_fonetico_b', udf_criaMetaphone(F.col('primeiro_nome_b')))
dfl = dfl.withColumn('ultimo_nome_fonetico_a', udf_criaMetaphone(F.col('ultimo_nome_a')))
dfl = dfl.withColumn('ultimo_nome_fonetico_b', udf_criaMetaphone(F.col('ultimo_nome_b')))

In [0]:
dfl = dfl.withColumn('nome_fonetico_a', udf_criaMetaphone(F.col('nome')))
dfl = dfl.withColumn('nome_fonetico_b', udf_criaMetaphone(F.col('nome_b')))
dfl = dfl.withColumn('nome_fonetico_mae_a', udf_criaMetaphone(F.col('mae')))
dfl = dfl.withColumn('nome_fonetico_mae_b', udf_criaMetaphone(F.col('mae_b')))

In [32]:
dfl.show(5)

+---+---------+--------+----------+--------------------+------+-----+---------------+--------------------+--------------+--------------+----+---------+-------------+-----------+--------------------+--------+--------------------+--------------------+----------------+--------------+---------------+---------------+-------------+-------------+------------------------+------------------------+----------------------+----------------------+---------------+---------------+-------------------+-------------------+
| id|      cep|  cidade| data_nasc|            endereco|estado|idade|            mae|                nome|tipo_sanguineo|           cpf|id_b|    cep_b|     cidade_b|data_nasc_b|          endereco_b|estado_b|               mae_b|              nome_b|tipo_sanguineo_b|         cpf_b|primeiro_nome_a|primeiro_nome_b|ultimo_nome_a|ultimo_nome_b|primeiro_nome_fonetico_a|primeiro_nome_fonetico_b|ultimo_nome_fonetico_a|ultimo_nome_fonetico_b|nome_fonetico_a|nome_fonetico_b|nome_fonetico_mae_a|n

In [33]:
dfl.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

## Criação da Função de Similaridade pelo Nome e Nome da Mãe com Jaro Winkler (fonético)


In [0]:
def compare(primeiro_nome_fonetico_a, ultimo_nome_fonetico_a, nome_fonetico_mae_a, primeiro_nome_fonetico_b, ultimo_nome_fonetico_b, nome_fonetico_mae_b):
  sim = 0
  
  #comparando atributos nominais
  sim = jellyfish.jaro_winkler(primeiro_nome_fonetico_a, primeiro_nome_fonetico_b)
  sim += jellyfish.jaro_winkler(ultimo_nome_fonetico_a, ultimo_nome_fonetico_b)
  sim += jellyfish.jaro_winkler(nome_fonetico_mae_a, nome_fonetico_mae_b)
  
  #Media aritmética simples
  sim = abs(float(sim/3))
  
  return sim

udf_compare = F.udf(compare, StringType())  

## Rodando Comparação


In [0]:
dfr = dfl.withColumn('similaridade', 
                     udf_compare(F.col('primeiro_nome_fonetico_a'),
                                 F.col('ultimo_nome_fonetico_a'),
                                 F.col('nome_fonetico_mae_a'),
                                 F.col('primeiro_nome_fonetico_b'),
                                 F.col('ultimo_nome_fonetico_b'),
                                 F.col('nome_fonetico_mae_b')))

In [59]:
dfr.printSchema()

root
 |-- id: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- id_b: string (nullable = true)
 |-- cep_b: string (nullable = true)
 |-- cidade_b: string (nullable = true)
 |-- data_nasc_b: string (nullable = true)
 |-- endereco_b: string (nullable = true)
 |-- estado_b: string (nullable = true)
 |-- mae_b: string (nullable = true)
 |-- nome_b: string (nullable = true)
 |-- tipo_sanguineo_b: string (nullable = true)
 |-- cpf_b: string (nullable = true)
 |-- primeiro_nome_a: string (nullable = true)
 |-- primeiro_nome_b: string (nullable = true)
 |-- ultimo_nome_a: string (nullable = true)
 |-- ultimo_nome_b: string (nullable = true

In [60]:
# Exibindo base de resultado
dfr.select(['id', 'id_b', 'nome', 'nome_b', 'cpf', 'cpf_b', 'similaridade']).show(10)

+---+----+--------------------+--------------------+--------------+--------------+-------------------+
| id|id_b|                nome|              nome_b|           cpf|         cpf_b|       similaridade|
+---+----+--------------------+--------------------+--------------+--------------+-------------------+
|  0|  01|Vicente Oliver Mo...|Felipe Bruno dos ...|097.627.958-49|506.865.202-46|                0.6|
|  0|  02|Vicente Oliver Mo...|Marcio Heitor Sil...|097.627.958-49|325.828.867-42| 0.5703703703703703|
|  0|  03|Vicente Oliver Mo...|Theo Mario Victor...|097.627.958-49|284.536.218-88|0.19166666666666668|
|  0|  04|Vicente Oliver Mo...|Pietro Henrique d...|097.627.958-49|390.375.925-20|               0.25|
|  0|  05|Vicente Oliver Mo...|Kaique Thiago de ...|097.627.958-49|756.005.387-40|               0.25|
|  0|  06|Vicente Oliver Mo...|Gabriel Erick Oli...|097.627.958-49|551.341.139-39|0.20502645502645503|
|  0|  07|Vicente Oliver Mo...|Luan Caue Anthony...|097.627.958-49|756.04

In [0]:
# Deduplicando a base (eliminando linhas duplicadas, ficando somente com a de maior similaridade)
dfc = dfr.orderBy(['similaridade'], ascending=False).dropDuplicates(['id_b']).orderBy(['id_b']).select(['id', 'id_b', 'nome','mae', 'nome_b', 'mae_b', 'cpf','cpf_b', 'similaridade'])


In [0]:
# Criação de coluna de comparação de CPFs
dfc = dfc.withColumn('CPFs_Iguais',F.col('cpf') == F.col('cpf_b'))

In [71]:
dfc.count()

143

In [78]:
dfc.show(10)

+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
| id|id_b|                nome|                 mae|              nome_b|               mae_b|           cpf|         cpf_b|      similaridade|CPFs_Iguais|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------+--------------+------------------+-----------+
| 14|  01|Felipe Bruno dos ...|Isabelle Marcia A...|Felipe Bruno dos ...|Isabelle Marcia A...|506.865.202-46|506.865.202-46|               1.0|       true|
| 15|  02|Marcio Heitor Sil...|Alana Isadora Jul...|Marcio Heitor Sil...|       Alana Isadora|325.828.867-42|325.828.867-42|0.9743589743589745|       true|
| 16|  03|Theo Mario Victor...|  Caroline Gabriela |Theo Mario Victor...|  Caroline Gabriela |284.536.218-88|284.536.218-88|               1.0|       true|
| 17|  04|Pietro Henry de P...|       Tereza Bruna |Pietro Henri

In [80]:
dfc.limit(100).toPandas()

,id,id_b,nome,mae,nome_b,mae_b,cpf,cpf_b,similaridade,CPFs_Iguais
0,14,01,Felipe Bruno dos Santos,Isabelle Marcia Antonella,Felipe Bruno dos Santos,Isabelle Marcia Antonella,506.865.202-46,506.865.202-46,1.0,True
1,15,02,Marcio Heitor Silveira,Alana Isadora Juliana,Marcio Heitor Silveira,Alana Isadora,325.828.867-42,325.828.867-42,0.9743589743589745,True
2,16,03,Theo Mario Victor Barbosa,Caroline Gabriela,Theo Mario Victor Barbosa,Caroline Gabriela,284.536.218-88,284.536.218-88,1.0,True
3,17,04,Pietro Henry de Paula,Tereza Bruna,Pietro Henrique de Paula,Tereza Bruna,390.375.925-20,390.375.925-20,1.0,True
4,18,05,Kaique Thiago de Paula,Alice Leticia,Kaique Thiago de Paula,Alice Leticia,756.005.387-40,756.005.387-40,1.0,True
5,19,06,Gabriel Erick Oliver da Luz,Helena Agatha Teresinha,Gabriel Erick Oliver da Luz,Elena Agatha Teresinha,551.341.139-39,551.341.139-39,0.9841269841269842,True
6,20,07,Luan Caue Anthony da Mota,Vanessa Camila Nicole,Luan Caue Anthony da Mota,Vanessa Camila Nicole,756.048.583-90,756.048.583-90,1.0,True
7,21,08,Enzo Diego Rocha,Isabelle Maite Rosa,Enzo D Rocha,Isabelle Maite Rosa,386.351.591-95,386.351.591-95,1.0,True
8,22,09,Cesar Matheus Caue Rodrigues,Sophie Bianca,Cesar Matheus Caue Rodrigues,Sophie Bianca,885.264.747-31,885.264.747-31,1.0,True
9,23,10,Caua Isaac Castro,Giovana Antonella Larissa,Caua Isaac Castro,Giovana Antonella Larissa,489.611.821-96,489.611.821-96,1.0,True


## Criação da Função de Similaridade pelo Nome e Nome da Mãe com Jaro Winkler (literal)


In [0]:
def compare_2(primeiro_nome_a, ultimo_nome_fonetico_a, nome_fonetico_mae_a, primeiro_nome_fonetico_b, ultimo_nome_fonetico_b, nome_fonetico_mae_b):
  sim = 0
  
  #comparando atributos nominais
  sim = jellyfish.jaro_winkler(primeiro_nome_a, primeiro_nome_b)
  sim += jellyfish.jaro_winkler(ultimo_nome_a, ultimo_nome_b)
  sim += jellyfish.jaro_winkler(nome_mae_a, nome_mae_b)
  
  #Media aritmética simples
  sim = abs(float(sim/3))
  
  return sim

udf_compare_2 = F.udf(compare_2, StringType())  

## Rodando Comparação


In [0]:
dfr_2 = dfl.withColumn('similaridade', 
                     udf_compare(F.col('primeiro_nome_fonetico_a'),
                                 F.col('ultimo_nome_fonetico_a'),
                                 F.col('nome_fonetico_mae_a'),
                                 F.col('primeiro_nome_fonetico_b'),
                                 F.col('ultimo_nome_fonetico_b'),
                                 F.col('nome_fonetico_mae_b')))

In [47]:
# Exibindo base de resultado
dfr_2.select(['id', 'id_b', 'nome', 'nome_b', 'cpf', 'cpf_b', 'similaridade']).show(10)

+---+----+--------------------+--------------------+--------------+--------------+-------------------+
| id|id_b|                nome|              nome_b|           cpf|         cpf_b|       similaridade|
+---+----+--------------------+--------------------+--------------+--------------+-------------------+
|  0|  01|Vicente Oliver Mo...|Felipe Bruno dos ...|097.627.958-49|506.865.202-46|                0.6|
|  0|  02|Vicente Oliver Mo...|Marcio Heitor Sil...|097.627.958-49|325.828.867-42| 0.5703703703703703|
|  0|  03|Vicente Oliver Mo...|Theo Mario Victor...|097.627.958-49|284.536.218-88|0.19166666666666668|
|  0|  04|Vicente Oliver Mo...|Pietro Henrique d...|097.627.958-49|390.375.925-20|               0.25|
|  0|  05|Vicente Oliver Mo...|Kaique Thiago de ...|097.627.958-49|756.005.387-40|               0.25|
|  0|  06|Vicente Oliver Mo...|Gabriel Erick Oli...|097.627.958-49|551.341.139-39|0.20502645502645503|
|  0|  07|Vicente Oliver Mo...|Luan Caue Anthony...|097.627.958-49|756.04

In [0]:
# Deduplicando a base (eliminando linhas duplicadas, ficando somente com a de maior similaridade)
dfc_2 = dfr_2.orderBy(['similaridade'], ascending=False).dropDuplicates(['id_b']).orderBy(['id_b']).select(['id', 'id_b', 'nome','nome_b','cpf','cpf_b', 'similaridade'])


In [0]:
# Criação de coluna de comparação de CPFs
dfc_2 = dfc_2.withColumn('CPFs_Iguais',F.col('cpf') == F.col('cpf_b'))

In [53]:
dfc_2.show(10)

+---+----+--------------------+--------------------+--------------+--------------+------------------+-----------+
| id|id_b|                nome|              nome_b|           cpf|         cpf_b|      similaridade|CPFs_Iguais|
+---+----+--------------------+--------------------+--------------+--------------+------------------+-----------+
| 14|  01|Felipe Bruno dos ...|Felipe Bruno dos ...|506.865.202-46|506.865.202-46|               1.0|       true|
| 15|  02|Marcio Heitor Sil...|Marcio Heitor Sil...|325.828.867-42|325.828.867-42|0.9743589743589745|       true|
| 16|  03|Theo Mario Victor...|Theo Mario Victor...|284.536.218-88|284.536.218-88|               1.0|       true|
| 17|  04|Pietro Henry de P...|Pietro Henrique d...|390.375.925-20|390.375.925-20|               1.0|       true|
| 18|  05|Kaique Thiago de ...|Kaique Thiago de ...|756.005.387-40|756.005.387-40|               1.0|       true|
| 19|  06|Gabriel Erick Oli...|Gabriel Erick Oli...|551.341.139-39|551.341.139-39|0.9841

In [76]:
dfc_2.limit(10).toPandas()

,id,id_b,nome,nome_b,cpf,cpf_b,similaridade,CPFs_Iguais
0,14,01,Felipe Bruno dos Santos,Felipe Bruno dos Santos,506.865.202-46,506.865.202-46,1.0,True
1,15,02,Marcio Heitor Silveira,Marcio Heitor Silveira,325.828.867-42,325.828.867-42,0.9743589743589745,True
2,16,03,Theo Mario Victor Barbosa,Theo Mario Victor Barbosa,284.536.218-88,284.536.218-88,1.0,True
3,17,04,Pietro Henry de Paula,Pietro Henrique de Paula,390.375.925-20,390.375.925-20,1.0,True
4,18,05,Kaique Thiago de Paula,Kaique Thiago de Paula,756.005.387-40,756.005.387-40,1.0,True
5,19,06,Gabriel Erick Oliver da Luz,Gabriel Erick Oliver da Luz,551.341.139-39,551.341.139-39,0.9841269841269842,True
6,20,07,Luan Caue Anthony da Mota,Luan Caue Anthony da Mota,756.048.583-90,756.048.583-90,1.0,True
7,21,08,Enzo Diego Rocha,Enzo D Rocha,386.351.591-95,386.351.591-95,1.0,True
8,22,09,Cesar Matheus Caue Rodrigues,Cesar Matheus Caue Rodrigues,885.264.747-31,885.264.747-31,1.0,True
9,23,10,Caua Isaac Castro,Caua Isaac Castro,489.611.821-96,489.611.821-96,1.0,True


## Criação da Função de Similaridade pelo Nome e Data de Nascimento com Jaro Winkler 

## Criação da Função de Similaridade pelo Primeiro e Ultimo Nome e Nome da Mãe com *** 

## Criação da Função de Similaridade por outros Campos

## Criação da Função de Similaridade pelo Nome e Data de Nascimento com Jaro Winkler 

## Criação da Função de Similaridade pelo Nome e Data de Nascimento com Jaro Winkler 

In [0]:
def compare_2(primeiro_nome_a, ultimo_nome_fonetico_a, nome_fonetico_mae_a, primeiro_nome_fonetico_b, ultimo_nome_fonetico_b, nome_fonetico_mae_b):
  sim = 0
  
  #comparando atributos nominais
  sim = jellyfish.jaro_winkler(primeiro_nome_a, primeiro_nome_b)
  sim += jellyfish.jaro_winkler(ultimo_nome_a, ultimo_nome_b)
  sim += jellyfish.jaro_winkler(nome_mae_a, nome_mae_b)
  
  #Media aritmética simples
  sim = abs(float(sim/3))
  
  return sim

udf_compare_2 = F.udf(compare_2, StringType())  